# Downloading and Loading Datasets

To lessen the work of clicking individual files to download, we use BeautifulSoup to scrape the filenames from the website.

Parent directory: https://sdge.sdsc.edu/data/sdge/

**Note:** Right now we have only downloaded files from 2020-08

In [1]:
from bs4 import BeautifulSoup
import requests

# Fetch the web page
url = "https://sdge.sdsc.edu/data/sdge/ens_gfs_001/2020-08/"
response = requests.get(url)
data = response.text

# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

In [2]:
# The files/filenames are organized in a table
# Extract them from the table
fns = []
for tr in soup.find('table').find_all('tr'):
    # print(tr)
    row = [url.text for url in tr.find_all('a')]
    fns.append(row[1])

# first 2 elements are not filenames
fns = fns[2:]
fns

['dfmnfdrs_202008152000Z.nc',
 'dfmnfdrs_202008170800Z.nc',
 'dfmnfdrs_202008172000Z.nc',
 'dfmnfdrs_202008180800Z.nc',
 'dfmnfdrs_202008182000Z.nc',
 'dfmnfdrs_202008190800Z.nc',
 'dfmnfdrs_202008192000Z.nc',
 'dfmnfdrs_202008200800Z.nc',
 'dfmnfdrs_202008202000Z.nc',
 'dfmnfdrs_202008210800Z.nc',
 'dfmnfdrs_202008212000Z.nc',
 'dfmnfdrs_202008220800Z.nc',
 'dfmnfdrs_202008222000Z.nc',
 'dfmnfdrs_202008230800Z.nc',
 'dfmnfdrs_202008232000Z.nc',
 'dfmnfdrs_202008240800Z.nc',
 'dfmnfdrs_202008242000Z.nc',
 'dfmnfdrs_202008250800Z.nc',
 'dfmnfdrs_202008252000Z.nc',
 'dfmnfdrs_202008260800Z.nc',
 'dfmnfdrs_202008262000Z.nc',
 'dfmnfdrs_202008270800Z.nc',
 'dfmnfdrs_202008272000Z.nc',
 'dfmnfdrs_202008280800Z.nc',
 'dfmnfdrs_202008282000Z.nc',
 'dfmnfdrs_202008290800Z.nc',
 'dfmnfdrs_202008292000Z.nc',
 'dfmnfdrs_202008300800Z.nc',
 'dfmnfdrs_202008302000Z.nc',
 'dfmnfdrs_202008310800Z.nc',
 'dfmnfdrs_202008312000Z.nc',
 'wrfout_d02_202008151600Z.nc',
 'wrfout_d02_202008170400Z.nc',
 'wrfo

In [3]:
# Get the download urls, which are basically parent directory + filename
# Files will be downloaded to the local "data" folder
urls = []
dest = []
for i in fns:
    temp = 'https://sdge.sdsc.edu/data/sdge/ens_gfs_001/2020-08/' + i
    urls.append(temp)

    temp = 'data/' + i
    dest.append(temp)

**The following 2 code cell downloads the files from an online website, do not run unless needed.**

In [4]:
import requests
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

# Normal loop that downloads files from urls
# args is a zip of urls and destinations
def download_url(args):
    t0 = time.time()
    url, fn = args[0], args[1]
    try:
        r = requests.get(url)
        with open(fn, 'wb') as f:
            f.write(r.content)
        return(url, time.time() - t0)
    except Exception as e:
        print('Exception in download_url():', e)

# Download multiple files in parallel 
def download_parallel(args):
    cpus = cpu_count()
    results = ThreadPool(cpus - 1).imap_unordered(download_url, args)
    # prints results of downloaded file and time taken
    for result in results:
        print('url:', result[0], 'time (s):', result[1])

In [ ]:
# Run the download code
inputs = zip(urls, dest)
download_parallel(inputs)

Now we have the .nc files downloaded, we open them using the xarray library.

.nc files have metadata, which is retained when opened as datasets. However, converting them into Pandas dataframes loses that. 

In [5]:
import xarray as xr
import pandas as pd 

In [6]:
# dest has all the file paths
dest

['data/dfmnfdrs_202008152000Z.nc',
 'data/dfmnfdrs_202008170800Z.nc',
 'data/dfmnfdrs_202008172000Z.nc',
 'data/dfmnfdrs_202008180800Z.nc',
 'data/dfmnfdrs_202008182000Z.nc',
 'data/dfmnfdrs_202008190800Z.nc',
 'data/dfmnfdrs_202008192000Z.nc',
 'data/dfmnfdrs_202008200800Z.nc',
 'data/dfmnfdrs_202008202000Z.nc',
 'data/dfmnfdrs_202008210800Z.nc',
 'data/dfmnfdrs_202008212000Z.nc',
 'data/dfmnfdrs_202008220800Z.nc',
 'data/dfmnfdrs_202008222000Z.nc',
 'data/dfmnfdrs_202008230800Z.nc',
 'data/dfmnfdrs_202008232000Z.nc',
 'data/dfmnfdrs_202008240800Z.nc',
 'data/dfmnfdrs_202008242000Z.nc',
 'data/dfmnfdrs_202008250800Z.nc',
 'data/dfmnfdrs_202008252000Z.nc',
 'data/dfmnfdrs_202008260800Z.nc',
 'data/dfmnfdrs_202008262000Z.nc',
 'data/dfmnfdrs_202008270800Z.nc',
 'data/dfmnfdrs_202008272000Z.nc',
 'data/dfmnfdrs_202008280800Z.nc',
 'data/dfmnfdrs_202008282000Z.nc',
 'data/dfmnfdrs_202008290800Z.nc',
 'data/dfmnfdrs_202008292000Z.nc',
 'data/dfmnfdrs_202008300800Z.nc',
 'data/dfmnfdrs_2020

In [7]:
# test: open one dataset
ds = xr.open_dataset(dest[0])

# Print variable names
print(ds.data_vars)

Data variables:
    latitude                            (south_north, west_east) float32 ...
    longitude                           (south_north, west_east) float32 ...
    YYYY                                (time) int32 ...
    MM                                  (time) int32 ...
    DD                                  (time) int32 ...
    HH                                  (time) int32 ...
    mean_wtd_moisture_1hr               (time, south_north, west_east) float32 ...
    mean_wtd_moisture_10hr              (time, south_north, west_east) float32 ...
    mean_wtd_moisture_100hr             (time, south_north, west_east) float32 ...
    mean_wtd_moisture_1000hr            (time, south_north, west_east) float32 ...
    accumulated_precipitation_amount    (time, south_north, west_east) float32 ...
    surface_downwelling_shortwave_flux  (time, south_north, west_east) float32 ...
    air_temperature_2m                  (time, south_north, west_east) float32 ...
    eastward_10m_wind

In [8]:
# converting ds to df
df = ds.to_dataframe()
df

latitude   longitude  YYYY  MM  \
south_north west_east time                                                   
0           0         2020-08-28 20:00:00  31.578758 -121.618530  2020   8   
                      2020-08-28 21:00:00  31.578758 -121.618530  2020   8   
                      2020-08-28 22:00:00  31.578758 -121.618530  2020   8   
                      2020-08-28 23:00:00  31.578758 -121.618530  2020   8   
                      2020-08-29 00:00:00  31.578758 -121.618530  2020   8   
...                                              ...         ...   ...  ..   
68          107       2020-09-04 12:00:00  35.212891 -114.639526  2020   8   
                      2020-09-04 13:00:00  35.212891 -114.639526  2020   8   
                      2020-09-04 14:00:00  35.212891 -114.639526  2020   8   
                      2020-09-04 15:00:00  35.212891 -114.639526  2020   8   
                      2020-09-04 16:00:00  35.212891 -114.639526  2020   8   

                                           DD  HH  mean_wtd_moisture_1hr  \
south_north west_east time                                                 
0           0         2020-08-28 20:00:00  15  12                    NaN   
                      2020-08-28 21:00:00  15  13                    NaN   
                      2020-08-28 22:00:00  15  14                    NaN   
                      2020-08-28 23:00:00  15  15                    NaN   
                      2020-08-29 00:00:00  15  16                    NaN   
...                                        ..  ..                    ...   
68          107       2020-09-04 12:00:00  22   4               0.059641   
                      2020-09-04 13:00:00  22   5               0.065596   
                      2020-09-04 14:00:00  22   6               0.071396   
                      2020-09-04 15:00:00  22   7               0.075047   
                      2020-09-04 16:00:00  22   8               0.080164   

                                           mean_wtd_moisture_10hr  \
south_north west_east time                                          
0           0         2020-08-28 20:00:00                     NaN   
                      2020-08-28 21:00:00                     NaN   
                      2020-08-28 22:00:00                     NaN   
                      2020-08-28 23:00:00                     NaN   
                      2020-08-29 00:00:00                     NaN   
...                                                           ...   
68          107       2020-09-04 12:00:00                0.076455   
                      2020-09-04 13:00:00                0.077377   
                      2020-09-04 14:00:00                0.078689   
                      2020-09-04 15:00:00                0.079960   
                      2020-09-04 16:00:00                0.081895   

                                           mean_wtd_moisture_100hr  \
south_north west_east time                                           
0           0         2020-08-28 20:00:00                      NaN   
                      2020-08-28 21:00:00                      NaN   
                      2020-08-28 22:00:00                      NaN   
                      2020-08-28 23:00:00                      NaN   
                      2020-08-29 00:00:00                      NaN   
...                                                            ...   
68          107       2020-09-04 12:00:00                 0.091188   
                      2020-09-04 13:00:00                 0.091657   
                      2020-09-04 14:00:00                 0.092322   
                      2020-09-04 15:00:00                 0.092624   
                      2020-09-04 16:00:00                 0.093931   

                                           mean_wtd_moisture_1000hr  ...  \
south_north west_east time                                           ...   
0           0         2020-08-28 20:00:00                       NaN  ...   
   

# Data Cleaning

some messy first look at the data, using that one ds we loaded ^

In [ ]:
ds.data_vars